In [1]:
#ls train/cats/cat.10171.jpg

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# img = load_img('train/cats/cat.10171.jpg')  # this is a PIL image
# x = img_to_array(img)  # this is a Numpy array with shape (300, 300, 3)
# x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 300, 300, 3)
# x.shape

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
# i = 0
# for batch in datagen.flow(x, batch_size=10,
#                          save_to_dir='preview', save_prefix='el', save_format='jpeg'):
#     i += 1
#     if i > 20:
#         break  # otherwise the generator would loop indefinitely
        
# #print(x.shape)

/home/mahlet/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# MODEL
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# COMPILE
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [4]:
batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   rescale=1./255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# this is a generator that will read pictures found in subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'dogscats/train',  # this is the target directory
        target_size=(100, 100),  # all images will be resized to 300x300
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'dogscats/valid',
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:

# TRAINING
model.fit_generator(
        train_generator,
        steps_per_epoch= batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps= batch_size)

model.save('50_epochs.h5')  # always save your weights after training or during training

Epoch 1/50
64/64 [==============================] - 154s 2s/step - loss: 0.6931 - acc: 0.5225 - val_loss: 0.6880 - val_acc: 0.5430
Epoch 2/50
64/64 [==============================] - 102s 2s/step - loss: 0.6785 - acc: 0.5676 - val_loss: 0.6789 - val_acc: 0.5427
Epoch 3/50
64/64 [==============================] - 112s 2s/step - loss: 0.6583 - acc: 0.6086 - val_loss: 0.6693 - val_acc: 0.5777
Epoch 4/50
64/64 [==============================] - 100s 2s/step - loss: 0.6631 - acc: 0.6032 - val_loss: 0.6684 - val_acc: 0.5683
Epoch 5/50
64/64 [==============================] - 107s 2s/step - loss: 0.6565 - acc: 0.6101 - val_loss: 0.6405 - val_acc: 0.6278
Epoch 6/50
64/64 [==============================] - 96s 1s/step - loss: 0.6346 - acc: 0.6362 - val_loss: 0.6861 - val_acc: 0.5765
Epoch 7/50
64/64 [==============================] - 106s 2s/step - loss: 0.6399 - acc: 0.6433 - val_loss: 0.6311 - val_acc: 0.6368
Epoch 8/50
64/64 [==============================] - 100s 2s/step - loss: 0.6323 - ac

In [6]:
model.load_weights('50_epochs.h5')

In [7]:
# import matplotlib as plt
# plt.numpy(o9s)

In [8]:
# from PIL import Image
# img = Image.open('images.jpeg')

In [9]:
# from array import array
# import cv2
# # a = array(img).reshape(1,200,200,1)
# img = cv2.imread('./test1/1.jpg')

In [10]:
ls keras_validation/dogs/index2.jpeg

keras_validation/dogs/index2.jpeg


In [11]:
# predicting images
#dogs 0
#cats 1
from PIL import Image
# from SimpleCV import *
from keras.preprocessing import image
import numpy as np
img = image.load_img('keras_validation/dogs/index2.jpeg', target_size=(100, 100))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict_classes(images, batch_size=10)
print(classes)

[[1]]
